# 1-3-FACSexpression
Jakke Neiro$^1$
1. Aboobaker laboraotry, Department of Zoology, University of Oxford



## Contents of the notebook
* 1. Introduction
* 2. Alignment
    * 2.1 Hisat alignment
    * 2.2 Salmon pseudoalingment
* 3. Coverage
* 4. Proportional TPM values
    * 4.1 Calculation of proportional values
    * 4.2 Inspection of proportional values

## Files
* Input:
    * oenalx1_1.fastq.gz, oenalx1_2.fastq.gz, oenalx2_1.fastq.gz, oenalx2_2.fastq.gz, oenalxins_1.fastq.gz, oenalxins_2.fastq.gz
    * sleuth_gene.csv
* Output:
    * /hydra/FACS/salmon_bootstrap
    * x1_mean_final.bw
    * FACS_prop.csv

# 1. Introduction

RNA-seq samples from X1, X2 and Xins compartments were used to generate coverage tracks and compartment-specific TPM values for X1, X2 and Xins.

# 2. Alignment

## 2.1 Hisat alignment

The Önal et al. (2012) X1, X2 and Xins RNA-seq samples were aligned to the genome. with Hisat2. The other RNA-seq samples had already been aligned in 1-1-Genomeannotation. 

In [1]:
%%bash
cd /hydra/neiro_bam/
less hisat2_align_oenal.sh

#!/bin/bash

~/software/hisat2-2.1.0/hisat2 -p 8 --dta --known-splicesite-infile smes_v2_hconf_hisat2_splice_sites -x ~/sexual_genome_annotation_files/sexual_genome_annotation/final_dd_Smed_g4_hisat2 -1 oenalx1_1.fastq.gz -2 oenalx1_2.fastq.gz -S oenalx1.sam

~/software/hisat2-2.1.0/hisat2 -p 8 --dta --known-splicesite-infile smes_v2_hconf_hisat2_splice_sites -x ~/sexual_genome_annotation_files/sexual_genome_annotation/final_dd_Smed_g4_hisat2 -1 oenalx2_1.fastq.gz -2 oenalx2_2.fastq.gz -S oenalx2.sam

~/software/hisat2-2.1.0/hisat2 -p 8 --dta --known-splicesite-infile smes_v2_hconf_hisat2_splice_sites -x ~/sexual_genome_annotation_files/sexual_genome_annotation/final_dd_Smed_g4_hisat2 -1 oenalxins_1.fastq.gz -2 oenalxins_2.fastq.gz -S oenalxins.sam



In [ ]:
#%%bash
#cd /hydra/neiro_bam/
#sudo nohup ./hisat2_align.sh &

The sam files were converted to bam files.

In [2]:
%%bash
cd /hydra/neiro_bam/
less hisat_samtools_oenal.sh

#!/bin/bash

samtools view -b -S oenalx1.sam > oenalx1.bam
samtools view -b -S oenalx2.sam > oenalx2.bam
samtools view -b -S oenalxins.sam > oenalxins.bam



In [ ]:
#%%bash
#cd /hydra/neiro_bam/
#sudo nohup ./hisat_samtools_oenal.sh &

The bam files were sorted:

In [14]:
#%%bash
#cd /hydra/neiro_bam/
#samtools sort -@ 8 -o oenalx1.sorted.bam oenalx1.bam
#samtools sort -@ 8 -o oenalx2.sorted.bam oenalx2.bam
#samtools sort -@ 8 -o oenalxins.sorted.bam oenalxins.bam

The aligned files were later used to generate coverage tracks (bigwigs).

## 2.2 Salmon pseudoalignment

First, the transcripts were extracted by using the new annotation: 

In [2]:
%%bash
cd /hydra/FACS
gffread -w stringtie_transcripts.fa -g ~/sexual_genome_annotation_files/sexual_genome_annotation/final_dd_Smed_g4.fa /hydra/sexual_genome_annotation_files/ncrna_Neiro/stringtie_merged.gtf

In [3]:
%%bash
cd /hydra/FACS
grep -c ">" stringtie_transcripts.fa
grep ">" stringtie_transcripts.fa | head -5

91068
>SMEST026673001.1
>MSTRG.1.2
>MSTRG.1.3
>MSTRG.2.1
>MSTRG.2.2


The transcripts were used to generate the salmon index: 

In [ ]:
%%bash
cd ../FACS
echo "#!/bin/bash" > salmon_index_stringtie.sh
echo "~/software/salmon/bin/salmon index -t ~/FACS/stringtie_transcripts.fa -i salmon_index_stringtie -k 31" >> salmon_index_stringtie.sh
chmod +x salmon_index_stringtie.sh
less salmon_index_stringtie.sh

In [ ]:
#%%bash
#cd /hydra/FACS
#./salmon_index_stringtie.sh

Salmon was used to pseudoalign samples to the transcripts: 

In [ ]:
%%bash
cd ../FACS
echo "#!/bin/bash" > salmon_quant_bootstrap.sh
echo "for f in ../../../../../data/ngs/smed/facs_rnaseq_datasets/sexual/anish_trimmed_files/*fastq.gz; do ~/software/salmon/bin/salmon quant -i salmon_index_stringtie -l A -r \$f --validateMappings -o salmon_bootstrap/\${f:77} --numBootstraps 50; done" >> salmon_quant_bootstrap.sh
chmod +x salmon_quant_bootstrap.sh
less salmon_quant_bootstrap.sh

In [ ]:
#%%bash
#cd ../FACS
#nohup ./salmon_quant_bootstrap.sh &

In [ ]:
#%%bash
#cd ../FACS
#nohup ~/software/salmon/bin/salmon quant -i salmon_index_stringtie -l A -1 ../../../../../data/ngs/smed/facs_rnaseq_datasets/sexual/neiro_bam/oenalx1_1.fastq.gz -2 ../../../../../data/ngs/smed/facs_rnaseq_datasets/sexual/neiro_bam/oenalx1_2.fastq.gz --validateMappings -o salmon_bootstrap/oenalx1 --numBootstraps 50 & 
#nohup ~/software/salmon/bin/salmon quant -i salmon_index_stringtie -l A -1 ../../../../../data/ngs/smed/facs_rnaseq_datasets/sexual/neiro_bam/oenalx2_1.fastq.gz -2 ../../../../../data/ngs/smed/facs_rnaseq_datasets/sexual/neiro_bam/oenalx2_2.fastq.gz --validateMappings -o salmon_bootstrap/oenalx2 --numBootstraps 50 &
#nohup ~/software/salmon/bin/salmon quant -i salmon_index_stringtie -l A -1 ../../../../../data/ngs/smed/facs_rnaseq_datasets/sexual/neiro_bam/oenalxins_1.fastq.gz -2 ../../../../../data/ngs/smed/facs_rnaseq_datasets/sexual/neiro_bam/oenalxins_2.fastq.gz --validateMappings -o salmon_bootstrap/oenalxins --numBootstraps 50 &

In [18]:
%%bash
cd /hydra/FACS/salmon_bootstrap
ls -l | wc -l

188


The names of all samples were included in a list: 

In [ ]:
%%bash
cd /hydra/FACS/salmon_bootstrap
ls -d rna* > rna_list_tpm.txt

In [16]:
%%bash
cd /hydra/FACS/salmon_bootstrap
cd rna_bartscherer_tail_foxd_000h_SRR1012785_1.fastq.trimmed.fastq.gz
ls

aux_info
cmd_info.json
libParams
lib_format_counts.json
logs
quant.sf


# 3. Coverage

The bam files for X1, X2 and Xins samples were used to generate coverage tracks (bigwig files).

In [ ]:
%%bash
cd /hydra/neiro_bam
echo "#!/bin/bash" > rna_coverage.sh
echo "cd ~/../../../../data/ngs/smed/facs_rnaseq_datasets/sexual/neiro_bam" >> rna_coverage.sh
echo "samtools index -b oenalx1.sorted.bam" >> rna_coverage.sh
echo "samtools index -b oenalx2.sorted.bam" >> rna_coverage.sh
echo "samtools index -b oenalxins.sorted.bam" >> rna_coverage.sh
echo "samtools index -b rna_pearson_x1_na_na_SRR2009674_1.fastq.trimmed.fastq.bam" >> rna_coverage.sh
echo "samtools index -b rna_pearson_x1_na_na_SRR2009675_1.fastq.trimmed.fastq.bam" >> rna_coverage.sh
echo "samtools index -b rna_pearson_x2_na_na_SRR2009676_1.fastq.trimmed.fastq.bam" >> rna_coverage.sh
echo "samtools index -b rna_pearson_x2_na_na_SRR2009677_1.fastq.trimmed.fastq.bam" >> rna_coverage.sh
echo "samtools index -b rna_reddien_x1_na_000h_SRR1302023_1.fastq.trimmed.fastq.bam" >> rna_coverage.sh
echo "samtools index -b rna_reddien_x1_na_000h_SRR1302024_1.fastq.trimmed.fastq.bam" >> rna_coverage.sh
echo "samtools index -b rna_reddien_x2_na_000h_SRR1302025_1.fastq.trimmed.fastq.bam" >> rna_coverage.sh
echo "samtools index -b rna_reddien_x2_na_000h_SRR1302026_1.fastq.trimmed.fastq.bam" >> rna_coverage.sh
echo "samtools index -b rna_sanchez_x1_na_na_SRR2407874_1.fastq.trimmed.fastq.bam" >> rna_coverage.sh
echo "samtools index -b rna_sanchez_x1_na_na_SRR2407875_1.fastq.trimmed.fastq.bam" >> rna_coverage.sh
echo "samtools index -b rna_sanchez_x1_na_na_SRR2407876_1.fastq.trimmed.fastq.bam" >> rna_coverage.sh
echo "samtools index -b rna_sanchez_x1_na_na_SRR2407877_1.fastq.trimmed.fastq.bam" >> rna_coverage.sh
echo "samtools index -b rna_sanchez_xins_na_na_SRR2407878_1.fastq.trimmed.fastq.bam" >> rna_coverage.sh
echo "samtools index -b rna_sanchez_xins_na_na_SRR2407879_1.fastq.trimmed.fastq.bam" >> rna_coverage.sh
echo "samtools index -b rna_sanchez_xins_na_na_SRR2407880_1.fastq.trimmed.fastq.bam" >> rna_coverage.sh
echo "samtools index -b rna_sanchez_xins_na_na_SRR2407881_1.fastq.trimmed.fastq.bam" >> rna_coverage.sh

echo "bamCoverage -b oenalx1.sorted.bam -o oenalx1.bw --normalizeUsingRPKM" >> rna_coverage.sh
echo "bamCoverage -b oenalx2.sorted.bam -o oenalx2.bw --normalizeUsingRPKM" >> rna_coverage.sh
echo "bamCoverage -b oenalxins.sorted.bam -o oenalxins.bw --normalizeUsingRPKM" >> rna_coverage.sh
echo "bamCoverage -b rna_pearson_x1_na_na_SRR2009674_1.fastq.trimmed.fastq.bam -o pearson_x1_1.bw --normalizeUsingRPKM" >> rna_coverage.sh
echo "bamCoverage -b rna_pearson_x1_na_na_SRR2009675_1.fastq.trimmed.fastq.bam -o pearson_x1_2.bw --normalizeUsingRPKM" >> rna_coverage.sh
echo "bamCoverage -b rna_pearson_x2_na_na_SRR2009676_1.fastq.trimmed.fastq.bam -o pearson_x2_1.bw --normalizeUsingRPKM" >> rna_coverage.sh
echo "bamCoverage -b rna_pearson_x2_na_na_SRR2009677_1.fastq.trimmed.fastq.bam -o pearson_x2_2.bw --normalizeUsingRPKM" >> rna_coverage.sh
echo "bamCoverage -b rna_reddien_x1_na_000h_SRR1302023_1.fastq.trimmed.fastq.bam -o reddien_x1_1.bw --normalizeUsingRPKM" >> rna_coverage.sh
echo "bamCoverage -b rna_reddien_x1_na_000h_SRR1302024_1.fastq.trimmed.fastq.bam -o reddien_x1_2.bw --normalizeUsingRPKM" >> rna_coverage.sh
echo "bamCoverage -b rna_reddien_x2_na_000h_SRR1302025_1.fastq.trimmed.fastq.bam -o reddien_x2_1.bw --normalizeUsingRPKM" >> rna_coverage.sh
echo "bamCoverage -b rna_reddien_x2_na_000h_SRR1302026_1.fastq.trimmed.fastq.bam -o reddien_x2_2.bw --normalizeUsingRPKM" >> rna_coverage.sh
echo "bamCoverage -b rna_sanchez_x1_na_na_SRR2407874_1.fastq.trimmed.fastq.bam -o sanchez_x1_1.bw --normalizeUsingRPKM" >> rna_coverage.sh
echo "bamCoverage -b rna_sanchez_x1_na_na_SRR2407875_1.fastq.trimmed.fastq.bam -o sanchez_x1_2.bw --normalizeUsingRPKM" >> rna_coverage.sh
echo "bamCoverage -b rna_sanchez_x1_na_na_SRR2407876_1.fastq.trimmed.fastq.bam -o sanchez_x1_3.bw --normalizeUsingRPKM" >> rna_coverage.sh
echo "bamCoverage -b rna_sanchez_x1_na_na_SRR2407877_1.fastq.trimmed.fastq.bam -o sanchez_x1_4.bw --normalizeUsingRPKM" >> rna_coverage.sh
echo "bamCoverage -b rna_sanchez_xins_na_na_SRR2407878_1.fastq.trimmed.fastq.bam -o sanchez_xins_1.bw --normalizeUsingRPKM" >> rna_coverage.sh
echo "bamCoverage -b rna_sanchez_xins_na_na_SRR2407879_1.fastq.trimmed.fastq.bam -o sanchez_xins_2.bw --normalizeUsingRPKM" >> rna_coverage.sh
echo "bamCoverage -b rna_sanchez_xins_na_na_SRR2407880_1.fastq.trimmed.fastq.bam -o sanchez_xins_3.bw --normalizeUsingRPKM" >> rna_coverage.sh
echo "bamCoverage -b rna_sanchez_xins_na_na_SRR2407881_1.fastq.trimmed.fastq.bam -o sanchez_xins_4.bw --normalizeUsingRPKM" >> rna_coverage.sh
chmod +x rna_coverage.sh

In [19]:
#%%bash
#cd /hydra/neiro_bam
#nohup ./coverage_rna.sh &

Wiggletools was used to generate a mean coverage track of all the samples: 

In [10]:
%%bash
cd /hydra/neiro_bam/
less coverage_mean_wiggle.sh

#!/bin/bash
wiggletools mean oenalx1.bw pearson_x1_1.bw pearson_x1_2.bw reddien_x1_1.bw reddien_x1_2.bw sanchez_x1_1.bw sanchez_x1_2.bw sanchez_x1_3.bw sanchez_x1_4.bw > x1_mean_final.wig

wiggletools mean oenalx2.bw pearson_x2_1.bw pearson_x2_2.bw reddien_x2_1.bw reddien_x2_2.bw > x2_mean_final.wig

wiggletools mean oenalxins.bw sanchez_xins_1.bw sanchez_xins_2.bw sanchez_xins_3.bw sanchez_xins_4.bw > xins_mean_final.bw 


In [11]:
%%bash
cd /hydra/neiro_bam/
ls x1_mean_final.wig

x1_mean_final.wig


In [ ]:
%%bash
cd /hydra/neiro_bam
wigToBigWig x1_mean_final.wig ~/sexual_genome_annotation_files/sexual_genome_annotation/sizes.genome x1_mean_final.bw

In [12]:
%%bash
cd /hydra/FACS
ls *bw

forward.sum.bw
reverse.sum.bw
x1_mean_final.bw


# 4. Proportional TPM values 

In order to find genes enriched in the X1 compartment, proportional TPM values were assigned to the X1, X2 and Xins.

## 4.1 Calculation of proportional values

In [10]:
import pandas as pd
sleuth_gene = pd.read_csv("/hydra/FACS/sleuth_gene.csv")

In [14]:
sleuth_gene = sleuth_gene.iloc[:,2:22]

In [16]:
sleuth_agg = sleuth_gene.groupby(['gene']).sum()

* X1: oenalx1, SRR2009674, SRR2009675, SRR1302023, SRR1302024, SRR2407874, SRR2407875, SRR2407876, SRR2407877
* X2: oenalx2, SRR2009676, SRR2009677, SRR1302025, SRR1302026
* Xins: oenalxins, SRR2407878, SRR2407879, SRR2407880, SRR2407881

In [18]:
x1_agg = sleuth_agg.iloc[:,[0,3, 4, 7,8, 11, 12, 13, 14]]
x2_agg = sleuth_agg.iloc[:,[1,5, 6, 9, 10]]
xins_agg = sleuth_agg.iloc[:,[2,15, 16, 17, 18]]

Means were calculated for each gene in each compartment:

In [ ]:
x1_agg['mean'] = x1_agg.mean(axis=1)
x2_agg['mean'] = x2_agg.mean(axis=1)
xins_agg['mean'] = xins_agg.mean(axis=1)

The proportional values were calculated by dividing the gene expression value in each compartment by the total sum.

In [20]:
tpm_prop = pd.DataFrame({"X1": x1_agg['mean'], "X2": x2_agg['mean'], "Xins": xins_agg["mean"]})
tpm_prop["total"] = tpm_prop.sum(axis=1)
tpm_prop["X1.prop"] = tpm_prop["X1"]/tpm_prop["total"]
tpm_prop["X2.prop"] = tpm_prop["X2"]/tpm_prop["total"]
tpm_prop["Xins.prop"] = tpm_prop["Xins"]/tpm_prop["total"]

The original Rink IDs were added to the table: 

In [21]:
gffcmp = pd.read_csv("/hydra/sexual_genome_annotation_files/ncrna_Neiro/gffcmp.stringtie_merged.gtf.tmap", sep="\t")

In [ ]:
rink_genes = []
for i in range(0, len(tpm_prop)):
    gene = tpm_prop.index[i]
    rink_gene = gffcmp[gffcmp["qry_gene_id"] == gene].iloc[0,0]
    if rink_gene != "-":
        rink_genes.append(rink_gene)
    else:
        rink_genes.append(gene)

In [ ]:
tpm_prop["Rink.ID"] = rink_genes

In [ ]:
tpm_prop.to_csv("/hydra/FACS/FACS_prop.csv")

## 4.2 Inspection of proportional values

The frequencies in different enrichment categories were calculated: 

In [20]:
import pandas as pd
tpm_prop = pd.read_csv("/hydra/FACS/FACS_prop.csv")

In [21]:
tpm_prop.head()

,gene,X1,X2,Xins,total,X1.prop,X2.prop,Xins.prop,Rink.ID
0,MSTRG.1,36.191056,25.270065,54.627823,116.088943,0.311753,0.217678,0.470569,SMESG000026673.1
1,MSTRG.10,1.213314,1.256480,0.385004,2.854797,0.425009,0.440129,0.134862,MSTRG.10
2,MSTRG.100,13.166939,22.418585,39.262879,74.848402,0.175915,0.299520,0.524565,SMESG000026500.1
3,MSTRG.1000,11.094835,6.709955,6.726305,24.531095,0.452276,0.273529,0.274195,SMESG000000245.1
4,MSTRG.10000,18.631952,49.453499,49.371953,117.457403,0.158627,0.421033,0.420339,SMESG000030021.1


Total number of genes: 

In [2]:
len(tpm_prop)

28003

X1 enrichment category:

In [3]:
len(tpm_prop[tpm_prop["X1.prop"] >= 0.5])

4376

X2 enrichment category: 

In [4]:
len(tpm_prop[tpm_prop["X2.prop"] >= 0.5])

5284

Xins enrichment category: 

In [5]:
len(tpm_prop[tpm_prop["Xins.prop"] >= 0.5])

7944

X1 and X2 enrichment category: 

In [6]:
len(tpm_prop[((tpm_prop["X1.prop"] + tpm_prop["X2.prop"]) >= 0.75) & (tpm_prop["X1.prop"] < 0.5) & (tpm_prop["X2.prop"] < 0.5)])

2992

X1 and Xins enrichment category:

In [7]:
len(tpm_prop[((tpm_prop["X1.prop"] + tpm_prop["Xins.prop"]) >= 0.75) & (tpm_prop["X1.prop"] < 0.5) & (tpm_prop["Xins.prop"] < 0.5)])

1251

X2 and Xins enrichment category: 

In [8]:
len(tpm_prop[((tpm_prop["X2.prop"] + tpm_prop["Xins.prop"]) >= 0.75) & (tpm_prop["X2.prop"] < 0.5) & (tpm_prop["Xins.prop"] < 0.5)])

2054

Ubiquitous enrichment category: 

In [9]:
28003-4376-5284-7944-2992-1251-2054

4102

* Ubiquitous: 4102
* X1: 4376
* X1 and X2: 2992
* X1 and Xins: 1251
* X2: 5284
* X2 and Xins: 2054
* Xins: 7944

# 5. Transcripts per gene

In [2]:
import pandas as pd
gffcmp = pd.read_csv("/hydra/sexual_genome_annotation_files/ncrna_Neiro/gffcmp.stringtie_merged.gtf.tmap", sep="\t")

In [9]:
import numpy as np
genes = list(pd.Categorical(gffcmp["qry_gene_id"]).categories)
counts = np.zeros(len(genes))

In [13]:
len(gffcmp[gffcmp.iloc[:,3] == genes[1]])

3

In [22]:
#for i in range(len(genes)):
#    counts[i] = len(gffcmp[gffcmp.iloc[:,3] == genes[i]])
#    if i % 1000 == 0:
#        print(i)

In [23]:
pd.DataFrame({"Genes": genes, "Counts": counts}).to_csv("/hydra/sexual_genome_annotation_files/ncrna_Neiro/genetranscriptcounts.csv", index=False)

In [25]:
counts = pd.read_csv("/hydra/sexual_genome_annotation_files/ncrna_Neiro/genetranscriptcounts.csv")

In [28]:
counts[counts.iloc[:,1] > 100]

,Genes,Counts
1458,MSTRG.1131,138.0
3991,MSTRG.13590,206.0
4094,MSTRG.13683,106.0
10894,MSTRG.19803,291.0
11907,MSTRG.20714,571.0
13789,MSTRG.22408,143.0
17004,MSTRG.3975,112.0
20328,MSTRG.6967,104.0


# 6 Plots

In [2]:
import pandas as pd
FACS_prop = pd.read_csv("/hydra/FACS/FACS_prop.csv")

In [18]:
FACS_prop

,gene,X1,X2,Xins,total,X1.prop,X2.prop,Xins.prop,Rink.ID
0,MSTRG.1,36.191056,25.270065,54.627823,116.088943,0.311753,0.217678,0.470569,SMESG000026673.1
1,MSTRG.10,1.213314,1.256480,0.385004,2.854797,0.425009,0.440129,0.134862,MSTRG.10
2,MSTRG.100,13.166939,22.418585,39.262879,74.848402,0.175915,0.299520,0.524565,SMESG000026500.1
3,MSTRG.1000,11.094835,6.709955,6.726305,24.531095,0.452276,0.273529,0.274195,SMESG000000245.1
4,MSTRG.10000,18.631952,49.453499,49.371953,117.457403,0.158627,0.421033,0.420339,SMESG000030021.1
...,...,...,...,...,...,...,...,...,...
27998,SMESG000081816.1,0.060211,0.128577,0.078985,0.267773,0.224859,0.480171,0.294970,SMESG000081816.1
27999,SMESG000081849.1,0.100407,0.144069,0.000000,0.244476,0.410704,0.589296,0.000000,SMESG000081849.1
28000,SMESG000081971.1,0.018778,0.057717,0.022917,0.099412,0.188893,0.580580,0.230527,SMESG000081971.1
28001,SMESG000081986.1,0.216758,0.360742,0.352566,0.930067,0.233057,0.387867,0.379076,SMESG000081986.1


In [8]:
Al

array([3.61910557e+01, 1.21331363e+00, 1.31669391e+01, ...,
       1.87782153e-02, 2.16758497e-01, 1.37140870e-01])

In [26]:
import plotly.figure_factory as ff
Al = FACS_prop["X1.prop"].to_numpy()
Cu = FACS_prop["X2.prop"].to_numpy()
Y = Y = 1 - Al - Cu
enthalpy = (Al - 0.5) * (Y - 0.5) * (Cu - 1)**2
enthalpy = 2.e6 * (Al - 0.01) * Cu * (Al - 0.52) * (Cu - 0.48) * (Y - 1)**2 - 5000
fig = ff.create_ternary_contour(np.array([Al, Y, Cu]), enthalpy,
                                pole_labels=['Al', 'Y', 'Cu'],
                                interp_mode='cartesian',
                                ncontours=20,
                                colorscale='Viridis',
                                showscale=True,
                                title='Mixing enthalpy of ternary alloy')
fig.show()

ValueError: The sum of coordinates should be 1 or 100 for all data points

In [27]:
Al

array([0.31175282, 0.42500866, 0.17591476, ..., 0.18889267, 0.23305687,
       0.32608633])

In [34]:
import plotly.figure_factory as ff
import numpy as np
Al, Cu = np.mgrid[0:1:7j, 0:1:7j]
Al, Cu = Al.ravel(), Cu.ravel()
mask = Al + Cu <= 1
Al, Cu = Al[mask], Cu[mask]
Y = 1 - Al - Cu

enthalpy = (Al - 0.5) * (Cu - 0.5) * (Y - 1)**2
fig = ff.create_ternary_contour(np.array([Al, Y, Cu]), enthalpy,
                                pole_labels=['Al', 'Y', 'Cu'],
                                ncontours=20,
                                coloring='lines',
                                showmarkers=True)
fig.show()

In [15]:
Al, Cu = np.mgrid[0:1:7j, 0:1:7j]
Al, Cu = Al.ravel(), Cu.ravel()
Al

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.16666667, 0.16666667, 0.16666667,
       0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.33333333,
       0.33333333, 0.33333333, 0.33333333, 0.33333333, 0.33333333,
       0.33333333, 0.5       , 0.5       , 0.5       , 0.5       ,
       0.5       , 0.5       , 0.5       , 0.66666667, 0.66666667,
       0.66666667, 0.66666667, 0.66666667, 0.66666667, 0.66666667,
       0.83333333, 0.83333333, 0.83333333, 0.83333333, 0.83333333,
       0.83333333, 0.83333333, 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        ])

# FINNISHED

In [20]:
%%bash
